In [1]:
# from newspaper import Article
import pandas as pd
import re
import torch
import json

In [2]:
def clean(x):
    return x.replace("(","").replace(")","").replace("{","").replace("}","").replace("[","").replace("]","").replace('"',"").replace("'","").replace("=","")

In [3]:
# def source(document):
#     with open(f"NIKL_NEWSPAPER(v1.0)/{document[:14]}.json","rb") as f:
#         file=json.load(f)
#     for x in file["document"]:
#         if x["id"]==document:
#             return "\n".join([y["form"] for y in x["paragraph"]])
def source(document):
    with open(f"NIKL_NEWSPAPER(v1.0)/{document[:14]}.json","rb") as f:
        file=json.load(f)
    for x in file["document"]:
        if x["id"]==document:
            return [y["form"] for y in x["paragraph"]]

In [4]:
with open("NIKL_SC.json","rb") as f:
    sum_file=json.load(f)

In [5]:
corpus1=[{"src":source(x["document_id"]),"tgt":x["topic_sentences"]} for x in sum_file["data"]]
corpus1[:3]

[{'src': ['與, 쟁점법안 대폭후퇴 끝 타결',
   '여야가 6일 임시국회 종료 이틀을 남기고 주요 쟁점법안 처리 문제를 일괄 타결했다. 이로써 지난해 12월 10일 개회 이래 4주 동안 야당의 본회의장 불법 점거와 국회의장의 질서유지권 발동 등으로 가파르게 대치했던 정국이 정상화됐다.',
   '▶A3·4·5·6면에 관련기사',
   '한나라당 홍준표, 민주당 원혜영, 선진과 창조의 모임 문국현 원내대표는 이날 국회 귀빈식당에서 원내대표 회담을 두 차례 열어 미디어 관계 법안 등 쟁점 법안의 처리 방안에 대한 10개 항에 합의했다.',
   '여야는 이견이 없는 민생법안 등 100여 건을 이번 임시국회에서 협의 처리하되 아직 상임위원회에 상정되지 않았거나 심의를 거치지 못한 법안은 9일 다시 임시국회를 열어 처리하기로 했다.',
   '그러나 여야는 이날 합의에서 상당수 쟁점법안의 처리 시기는 물론이고 법안 상정 여부조차 정하지 못해 2월 1일 열리는 임시국회에서 이를 둘러싼 ‘제2차 입법전쟁’이 벌어질 가능성이 높다.',
   '여야는 최대 쟁점이었던 미디어 관계 법안 8건 중 언론중재법 등 여야 간 이견이 없는 법안 2건만 이번 임시국회에서 협의 처리하고 신문·방송 겸영 허용과 대기업 방송 진출 허용 등 여야가 맞서는 6개 법안은 이른 시일 안에 합의 처리하도록 노력하기로 했다.',
   '한미 자유무역협정(FTA) 비준동의안은 미국 차기 행정부가 출범(이달 20일)한 뒤 이른 시일 안에 협의 처리하기로 했다.',
   '출자총액제한제도 폐지 법안은 이번 임시국회에서 상임위원회에 상정하되 2월 임시국회에서 협의 처리하고 금산분리 완화 법안 역시 이번 임시국회에서 상정하되 여야가 합의 처리하도록 노력하기로 했다.',
   '재외국민에게 대선과 총선의 투표권을 부여하는 내용의 공직선거법 개정안은 여야 동수로 정치개혁특별위원회를 구성해 2월 임시국회에서 합의 처리하기로 했다.',
   '이에 앞서 민주당은 이날 오전 국회 본회의장과 행정안전위, 정무위

In [6]:
def load_jsonl(input_path):
    """
    Read list of objects from a JSON lines file.
    """
    data = []
    with open(input_path, 'r', encoding='utf-8') as f:
        for line in f:
            data.append(json.loads(line.rstrip('\n|\r')))
    print('Loaded {} records from {}'.format(len(data), input_path))
    return data
train=load_jsonl("train.jsonl")

Loaded 42803 records from train.jsonl


In [7]:
corpus2=[{"src":x["article_original"],"tgt":[x["article_original"][i] for i in x["extractive"]]} for x in train]
print(corpus2[:2])

[{'src': ['당진시 문화관광과를 대상으로 하는 행정사무감사에서 당진시립합창단 관계자가 보낸 것으로 추정되는 문자와 관련해 당진시의회가 행정사무조사특별위원회를 구성해 조사하겠다고 밝혔다.', '당진시의회 행정사무감사 3일차였던 지난 6일 문화관광과를 대상으로 하는 행감에서 최창용 의원은 “(당진시립합창단 정기연주회를 앞두고) 문자메세지를 제보받았다”며 “(음향팀에 보낸 것으로 추정되는) 해당 문자에는 ‘합창단이 소리를 작게 낼 것이니 알고 있으라’는 내용이었다”고 말했다.', '이어 “공연에서 소리를 작게 낸다는 것은 합창단으로서 그 임무를 하지 않겠다는 것”이며 “공연 자체를 무력화해 당진시를 망신 주려는 행위”라며 해당 문자를 보낸 단원 등 연루된 사람들을 찾아 사실관계를 확인하고 징계 등 책임을 물어야 한다고 지적했다.', '문제의 문자메세지를 전달받은 문화관광과는 감사법무담당관에게 조사를 의뢰했다.', '권경선 문화관광과장은 “누가, 어떻게 해서 해당 문자가 나온 것인지 정확히 조사해봐야 알 수 있다”며 “전달받은 문자 내용도 최 의원이 언급한 부분만 있어, 중간 내용만을 가지고는 전체를 유추할 수 없다”고 전했다.', '하지만 감사법무담당관실에서 아직 조사가 이뤄지지 않고 있어 당진시가 사태의 심각성을 인지하지 못하고 있다는 지적이다.', '그동안 행정사무감사가 진행되고 있어 사태를 지켜봤다던 감사법무담당관실에서는 “관계된 사람들을 조사해 사태를 파악해야 하는데, 아직 조사에 대한 뚜렷한 계획이 없다”고 답했다.', '한편 행감이 끝난 지난 12일 당진시의회에서는 당진시립합창단 문제를 비롯해 구체적인 조사가 필요한 부분에 대해 행정사무조사특별위원회를 구성해 운영하겠다고 밝혔다.', '김기재 의장은 “본회의 의결과 제적의원 1/3의 발의가 있으면 행정사무조사특별위원회를 구성할 수 있다”며 “다음 달 초 위원들과 상의해 위원회를 구성한다면 당진시립합창단 관련 사안을 비롯해 사회복지기관 위수탁 등에 대해 다룰 계획”이라고 말했다.'], 'tgt': [

In [8]:
corpus=corpus1+corpus2
corpus[:3]

[{'src': ['與, 쟁점법안 대폭후퇴 끝 타결',
   '여야가 6일 임시국회 종료 이틀을 남기고 주요 쟁점법안 처리 문제를 일괄 타결했다. 이로써 지난해 12월 10일 개회 이래 4주 동안 야당의 본회의장 불법 점거와 국회의장의 질서유지권 발동 등으로 가파르게 대치했던 정국이 정상화됐다.',
   '▶A3·4·5·6면에 관련기사',
   '한나라당 홍준표, 민주당 원혜영, 선진과 창조의 모임 문국현 원내대표는 이날 국회 귀빈식당에서 원내대표 회담을 두 차례 열어 미디어 관계 법안 등 쟁점 법안의 처리 방안에 대한 10개 항에 합의했다.',
   '여야는 이견이 없는 민생법안 등 100여 건을 이번 임시국회에서 협의 처리하되 아직 상임위원회에 상정되지 않았거나 심의를 거치지 못한 법안은 9일 다시 임시국회를 열어 처리하기로 했다.',
   '그러나 여야는 이날 합의에서 상당수 쟁점법안의 처리 시기는 물론이고 법안 상정 여부조차 정하지 못해 2월 1일 열리는 임시국회에서 이를 둘러싼 ‘제2차 입법전쟁’이 벌어질 가능성이 높다.',
   '여야는 최대 쟁점이었던 미디어 관계 법안 8건 중 언론중재법 등 여야 간 이견이 없는 법안 2건만 이번 임시국회에서 협의 처리하고 신문·방송 겸영 허용과 대기업 방송 진출 허용 등 여야가 맞서는 6개 법안은 이른 시일 안에 합의 처리하도록 노력하기로 했다.',
   '한미 자유무역협정(FTA) 비준동의안은 미국 차기 행정부가 출범(이달 20일)한 뒤 이른 시일 안에 협의 처리하기로 했다.',
   '출자총액제한제도 폐지 법안은 이번 임시국회에서 상임위원회에 상정하되 2월 임시국회에서 협의 처리하고 금산분리 완화 법안 역시 이번 임시국회에서 상정하되 여야가 합의 처리하도록 노력하기로 했다.',
   '재외국민에게 대선과 총선의 투표권을 부여하는 내용의 공직선거법 개정안은 여야 동수로 정치개혁특별위원회를 구성해 2월 임시국회에서 합의 처리하기로 했다.',
   '이에 앞서 민주당은 이날 오전 국회 본회의장과 행정안전위, 정무위

In [ ]:
from konlpy.tag import Okt
def load_data(article):
    okt=Okt()
    if (article["src"]) and (article["tgt"]):
        src_txt=article["src"]
        tgt_txt=article["tgt"]

        source=[okt.morphs(clean(sen)) for sen in src_txt]
        tgt=[okt.morphs(clean(sen)) for sen in tgt_txt]
    
        return {'src': source, 'tgt': tgt}

def format_to_lines(corpus):
    for k in range(-(-len(corpus))//4000):
        ls=[]
        for x in corpus[k*4000:(k+1)*4000]:
            if load_data(x) is not None:
                ls.append(load_data(x))
        with open(f"cnndm_sample.train.{k}.json","w") as file:
            json.dump(ls,file)
#     df=pd.read_csv(corpus_type+"_data.csv")
#     df.apply(_format_to_lines,axis=1).to_json("cnndm_sample.train.0"+".json",orient="records")
format_to_lines(corpus)

In [36]:
# df=pd.DataFrame([{"src":source(x["document_id"]),"tgt":" ".join(x["topic_sentences"])} for x in file["data"]])
# df

,src,tgt
0,"與, 쟁점법안 대폭후퇴 끝 타결\n여야가 6일 임시국회 종료 이틀을 남기고 주요 쟁...",여야가 6일 임시국회 종료 이틀을 남기고 주요 쟁점법안 처리 문제를 일괄 타결했다....
1,"[크로스 미디어]“장보고號 힘내세요 우리들이 있잖아요”\n“박사님, 항해 힘드셔도 ...",지난해 12월 중순 카리브 해에 있는 바하마를 떠난 장보고호는 이달 2일 미국령 푸...
2,산은-한화 ‘대우조선 신경전’ 팽팽\n“대우조선해양 인수대금 조달 계획 수정안 다시...,대우조선해양 최대 주주이자 매각 주간사회사인 산업은행과 대우조선 우선협상대상자인 한...
3,금융시장 ‘제2 리먼쇼크’ 아슬아슬\n《 미국의 대규모 공적자금 투입과 각국의 유동...,《 미국의 대규모 공적자금 투입과 각국의 유동성 공급으로 한동안 안정을 되찾았던 국...
4,[반짝반짝 2009 샛별]남자피겨 희망 13세 이동원\n“연아 누나가 여자 피겨의 ...,“연아 누나가 여자 피겨의 인기를 살렸듯이 저도 남자 피겨 붐을 일으키고 싶어요.”...
...,...,...
4384,"“치매센터 다닌 엄마, 몰라보게 좋아져”\n서울 데이케어센터 10곳 운영 호평\n“...",창전데이케어센터는 2015년 8월 서울시가 ‘서울형 치매 전용 데이케어센터’ 시범사...
4385,하성용 전 사장 구속기소…KAI 수사 일단락\n국내 최대 방산업체 회계분식·횡령·채...,검찰이 국내 최대 방산업체인 한국항공우주산업(KAI·카이)의 하성용(66) 전 사장...
4386,트럼프 국회연설서 “대북 압박 촉구”…협상 메시지도 담을까\n다음달 7~8일 방한·...,도널드 트럼프 미국 대통령이 다음달 7~8일 한국 방문 과정에서 국회 연설을 할 것...
4387,6월 서울 아파트 거래량 역대 최고\n[한겨레] 26일 현재 1만1735건 매매신고...,정부의 ‘6·19 부동산 대책’과 대대적인 투기 단속에도 6월 서울 아파트 매매 거...


In [70]:
# df.dropna(inplace=True)
# df.to_csv("train_data.csv",index=False)

In [28]:
# text=['코로나19 신규 환자 수가 64명 추가돼 이달 들어 나흘 연속 두 자릿수를 기록했습니다. 그러나 지역사회에서의 집단 감염이 계속되고 있어 방역 당국은 여전히 긴정하고 있습니다. 이덕영 기자입니다. 오늘 0시 기준 코로나19 신규 확진 환자는 64명입니다. 국내발생 47명 해외 유입 17명입니다. 이달 들어 나흘 연속 하루 신규 환자수가 두 자릿수를 나타내고 있는 겁니다. 하지만 추석 연휴를 받아 진단검사 수가 줄어든 가운데 전국적인 인구 이동으로 조용한 전파가 이뤄스 가능성도 있어 방역당국은 여전히 긴장을 늦추지 못하고 있습니다. 지역별로는 서울과 인천 경기 등 수도권에서 38명 부산 5명 경북 4명 대구 대전 충북 100명 등이 양성 판정을 받았습니다. 종교시설과 요양원 의료기관 등 사양한 장소에서의 집단발생이 계속되고 있는 상황입니다. 어제 정오 기준 서울 도봉구 단화병원 관련 확진자는 46명으로 늘어났고 경기도 포천의 소망공동체 유항원 관련해서도 14명이 확진됐습니다. 또 인천 미추홀구 소망교의 교인 10명과 부산 연재구 건강용품 4명의 관련 24명이 확진 판정을 받았습니다. 추석 귀성의 영향으로 부산을 방문한 서울과 울산 거주자 2명이 확진됐습니다. 한편 위중중증 환자는 105명이고 사망자는 1명이 추가돼 모두 421명으로 늘어났습니다. 엠비씨뉴스 이덕영입니다.', 
#  '국내 코로나19 신규 환자 수가 이틀 연속 예순명을 넘었습니다. 수도권과 광주 대전 등에서 확산세가 이어지고 있고 해외 유입 환자도 11일 만에 최대치를 기록했습니다. 김성현 기자입니다. 국내 코로나19 신규 확진 환자가 이틀 연속 60명대를 기록했습니다. 중앙방역대책본부는 오늘 0시 기준 신규 환자가 63명 추가돼 누적 환자 수가 1만3천30명으로 집계됐다고 밝혔습니다. 신규 환자 63명 중 지역 발생은 36명입니다. 지역별로는 경기가 15명 서울이 6명으로 3분의 이 가까이가 수도권에서 나왔습니다. 경기 의정부 장황마파체에선 주민과 1수장 이용자 등 지금까지 모두 22명이 확진 판정을 받았습니다. 수도권교회와 방문판매업소 등 기존 집단감염 사례를 중심으로 추가감염도 잇따르고 있습니다. 광주와 대전 충북 등 비수도권의 확산세도 기속되고 있습니다. 특히 광주 지역 사찰 광륵사에서 시작된 집단감염은 오피스텔과 교회 요양원까지 번지면서 누적 환자는 57명으로 늘었습니다. 해외 유입 환자도 27명 늘어 지난달 24는 이후 최대치를 기록했습니다. 사망자도 하루 새 1명 늘면서 지금까지 누적 사망자는 모두 283명으로 집계됐습니다. 엠비씨뉴스 김상한입니다.', 
#  '여러분 안녕하십니까 1월 5일 월요일 엠비씨 정오뉴스입니다. 코로나19 신규 확진 환자가 73명 추가되며 닷새 연속 두 자릿수를 유지했습니다. 하지만 추석 연휴 대규모 인구 이동으로 인한 조용한 전파가 발생했을 가능성을 배제할 수 없어 아직 안심하기 늘러 보입니다. 정동훈 기자가 보도합니다. 오늘 0시 기준 코로나19 신규 확진자는 73명 지역 발생이 64명 해외 유입 사례가 9명입니다. 지역별로 보면 경기 25명 서울 19명 인천 5명 등 수도권이 49명으로 여전히 대닫습니다. 주요 감염 사례는 코로나19에 취약한 의료기간과 요양시설 등에서 잇따랐습니다. 경기 부천 처음 요양과 관련해 11명이 확진됐고 부산 온종합병원 사례에서는 4명이 양성 판정을 받았습니다. 경기 포천 6건부대에서 12명이 무두기 확진 판정을 받았고 양평 건수럽 근로자들 이 외에도 서울 마포구 서울 디자인고 등기에서 집단감염이 확인됐습니다. 위 중증으로 당태가 악화한 환자는 2명 늘었고 특 차망자도 1명 늘었습니다. 다행히 하루 확진 환자수는 닷새째 두 자릿수를 유지하고 있는 사람. 방역 당국은 그러나 추석 여족 검사랑 감소와 이동량 증가했다는 확산 가능성을 고려하면 낙관하긴 이르다고 밝혔습니다. 이에 따라 정부는 오는 11일까지인 2단계 거리두기 신조체의 연장여부를 이번 주 준반까지 확단 초이를 지켜본 뒤 결정하기로 했습니다. 엠비씨뉴스 정동훈입니다.', 
#  '코로나19 신규 확진 환자가 97명 추가됐습니다. 6일 만에 하루 신규 환자 수가 두 자릿수로 떨어졌지만 많은 임파가 보였던 핼러윈데의 영향이 경수가 될 것을 보입니다. 이덕영 기자가 보도합니다. 중앙방역대책본부는 오늘 0시 기준 코로나19 확진 환자가 97명 추가돼 모두 2만6천732명으로 늘어났다고 밝혔습니다 신규 확진 환자 중 국내 발생은 79명 해외 유입은 18명이었습니다. 지역별로는 서울과 인천 경기 등 수도권에서 전체 확진자의 70%이 68명이 확진 판정을 받았습니다. 또 충남에서 11명 대구에서 3명이 확진됐고 울산과 강원 충부 전남 경북 경남 등에서도 확진이 잇따랐습니다. 지난달 27일 이후 엿새만에 하루 확진자 수가 두 자릿수를 나타냈지만 아직 안심할 수는 없는 상황입니다. 주말이라 진단검사 수가 적었던 데다 지난달 30일 일 해러인 데이를 맞아 서울 이태원 등에 많은 이파가 몰렀기 때문입니다. 텔러윈데이로 인한 감염 확산 있었는지 확인하기 위해선 앞으로 2주간 확진 추세를 더 지켜봐야 합니다. 한편 현재 3단계인 사회적 거리 두기를 5단계로 세분화해 적용하는 방안이 이번 주말부터 시행됩니다. 지역 유행과 전국 유행이 국은되고 각단계 적용이 수도권과 충천 건 고난 건등 7개 권역으로 차별화됩니다. 정부는 또 연말 건강검진의 사람들이 걸리는 걸 막기 위해 내년 6월까지로 검진 기간을 연장하는 방안을 검토하고 있습니다. 엠비씨뉴스 이덕영입니다.', 
#  '수도권과 대전에서의 집단감염이 이어지면서 어제 하루 코로나19 확진 환자 28명이 추가됐습니다. 정부는 부족한 병상을 확보하기 위해 오늘부터 무증상 환자는 1후 격리 해제가 도록했습니다. 이덕영 기자가 보도합니다. 오늘 오전 0시 기준 코로나19 확진 환자는 28명이 추가돼 모두 12563명으로 늘어났습니다. 하루 확진자 수가 30명대 아래로 내려온 건 지난 22일 이후 사흘 만입니다. 신규 확진 환자 중 지역에서 발생한 환자는 23명이고 해외에서 유입된 환자는 5명입니다. 지역별로는 서울 인천 경기 등 수도권에서 19명이 확진 판정을 받아 가장 많은 환자가 발생했습니다. 리치웨이 관련 환자가 25명으로 늘어나고 자동차 동호회와 관련해서도 5명의 환자가 확인되는 등 집단감염이 몸추지 않고 있기 때문입니다. 또 대전 방문판매업체 발집단 감염도 계속돼 대전에서 4명 충남에서 1명이 확진 판정을 받았습니다. 사망자는 1명이 추가돼 모두 282명으로 늘어났습니다. 한편 코로나19 중증 환자가 최근 여를 사이 2배 가까이 늘면서 병상 부적사태가 누나프로 다가오고 있습니다. 이에 따라 정부는 오늘부터 입원 중이 코로나19 환자의 격리해제 기준을 완화합니다. 증상이 나타난 지 10일으 여리 없이 상태가 호전되면 진단검사를 받지 않고도 격리가 해제될 수 있습니다. 또 무증상 환자의 경우 열흘 뒤 자동으로 격리가 해제됩니다. 이와 함께 정부는 충천권의 병상공동화병대 응체계 부칙과 공동생활치료센터 설치를 검토하고 있습니다. 엠비씨뉴스 이덕영입니다.', 
#  '어제 하루 발생한 코로나19 신규 확진자 수는 48명으로 확인됐습니다. 수도권과 대전을 중심으로 한 집단감염 여파로 지역발생 환자가 40명에 달했지만 최근 급증세였던 해외 유입 확진자는 한 자릿수로 줄었습니다. 이덕용 기자입니다. 오늘 오전 0시 기준 코로나19 확진 환자는 48명이 추가돼 모두 12421명으로 늘어났습니다. 이달 들어 처음으로 60명 넘는 신규 환자가 나왔던 하루 전에 비해 다소 줄어든 수치입니다. 하지만 신규 확진 환자 중 지역발생 환자는 40명으로 오히려 어제보다 4명 늘어났습니다. 반면 이달 들어 급증세를 모인 해외 유입 환자는 8명이 그처 다시 한 자릿수로 떨어졌습니다. 지역별로는 서울에서 17명 경기에서 10명으로 수도권이 가장 큰 증가폭을 나타냈습니다. 방문판매업체발 집단감염이 계속되고 있는 대전에서도 10명이 추가로 확진 판정을 받았습니다. 이밖에 강원에서 2명 대구 광주 세종 충남 경남에서 각 1명이 확진됐습니다. 수도권에서 시작된 엔차 집단감염이 머추지 않고 있는 가운데 다른 지역으로의 확산까지 나타나고 있는 상황 이에 따라 수도권을 중심으로 입원환자 병상이 부족해질 수 있다는 우려도 나오고 있습니다. 중앙 임상 이원에는 오늘 오후 국립 중앙으로나서 기자 회견을 갔고 이태운 기준 병경과 병상자원 관리방화 등에 대한 공고한을 발표할 예정입니다. 엠비씨뉴스 이덕영입니다.', 
#  '네 남은 코로나19 소식입니다. 어제 하루 신규 확진자가 60명 후반대로 급증했습니다. 국내 집단감염본만 아니라 해외 유입 사례까지 빠르게 늘고 있어 방역 당국은 비상입니다. 정동훈 기자입니다. 어제 하루 신규 확진 판정을 받은 국내 코로나19 환자는 모두 67명입니다. 지난 5월 28일 79명을 기록한 후 23일 만에 최대치입니다. 신규 확진 67명 중 지역 발생은 36명 서울 13명 경기 10명 인천 4명 등 수도권에서만 27명의 확진자가 나왔고 집단감염이 발생한 대전에서 5명 대구와 충남에서 각각 2명의 환자가 수도권과 대전 방문판매업체 종교시설동을 내기로 한 집단감염이 주변으로 계속 퍼지고 있는 겁니다. 어제 하루 해외에서 유입된 확진 환자는 31명 지난 4월 5일 40명을 기록한 이후 가장 많은 수치입니다. 국내 집단발병 사태가 확산하는 가운데 해외 유입 감염 사례까지 늘은 하면서 방역당국으로선 두 마리터키를 경시에 자발야 하는 상황에 됐습니다. 방역 당국은 최근 감염자가 늘고 있는 동남아1대에서 논천 이 소로 도기 위해 입국하는 외국인 노동자들이 증가하고 있는 만큼 대책을 구심하고 있습니다. 특히 방역 담국은 어제 방글라데시에서 같은 비행기로 입국한 내외국인 9명이 무두기로 양성 판정을 받은 것과 관련해 추가 확진자가 나올 가능성을 열어두고 상황을 예의 주시하고 있습니다. 엠비씨뉴스 정동훈입니다.', 
#  '코로나19 확진자가 사흘 연속 두 자릿수를 나타냈고 확진자 규모도 계속 줄고 있습니다. 중앙방역대책본부는 오늘 0시 기준 코로나19 신규 확진자가 61명 늘어나 누적 확진자는 2316명이라고 밝혔습니다. 신규 확진자의 감염경로를 보면 해외에서 유입된 10명을 제외한 51명이 국내에서 감염이 됐는데 서울 20명 경기 14명 인천 2명 등 수도권에서 36명의 확진자가 나왔습니다. 수도권 이외 지역에서는 부산 5명 경북 4명 강원과 충북 각 2명 울산과 경남 11명입니다. 코로나19 사망자는 3명 더 늘어 모두 388명이 됐고 위 중증 환자도 3명이 늘어 현재 144명입니다.', 
#  '정부가 코로나19 장기 전에 대비하기 위해서 사회적 거리두 위해 단계와 그에 따른 방역 조치를 개편합니다. 정부는 오늘 중앙재난안전대책본부 회의를 열고 현재 세 단계로 돼 있는 거리두기 단계를 개편한 방안을 확정해 오후 4시반 정례 브리핑을 통해 발표할 예정입니다. 이번 개편화는 거리두기 단계가 전국의 폐결적으로 적용되고 방역수에도 높아 사회적인 비용이 과도하게 된다는 지적을 감안해 지역은 물론 시설 업종별로 정교하고 셈이란 방역이 이뤄지록 가는 내용을 다물 것으로 예상됩니다.']
# tgt=[". ".join(x.split(". ")[:3]) for x in text]

In [22]:
# pd.DataFrame({"src":text,"tgt":tgt}).to_csv("test_data.csv",index=False)

In [27]:
# from konlpy.tag import Kkma
# from konlpy.tag import Okt
# def load_data(row):
#     kkma=Kkma()
#     okt=Okt()
#     article=row["src"]
#     abstract=row["tgt"]
# #     article=article.text
# #     abstract=tgt_text
#     src_txt=kkma.sentences(article)
#     tgt_txt=kkma.sentences(abstract)

#     source=[kkma.morphs(clean(sen)) for sen in src_txt]
#     tgt=[kkma.morphs(clean(sen)) for sen in tgt_txt]
# #     source=[okt.morphs(clean(sen)) for sen in src_txt]
# #     tgt=[okt.morphs(clean(sen)) for sen in tgt_txt]
#     return source,tgt
# def _format_to_lines(f):
#     source, tgt = load_data(f)
#     return {'src': source, 'tgt': tgt}

# def format_to_lines(mode):
#     corpus_type=mode
#     df=pd.read_csv(corpus_type+"_data.csv")
#     df.apply(_format_to_lines,axis=1).to_json("cnndm_sample."+corpus_type+".0"+".json",orient="records")
# format_to_lines("test")